# Nuclear segmentation

Using the DAPI staining and cellpose

In [24]:
import imageio as io
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import tifffile
from tqdm.notebook import tqdm
import pathlib
from cellpose import models, core
from cellpose.contrib.distributed_segmentation import numpy_array_to_zarr
import json
from pyometiff import OMETIFFReader
import zarr
import scanpy as sc

## Read in Xenium DAPI

In this part we import the DAPI OME TIFF, create a max projection of the different layers.

In [2]:
experiment_path = '/mnt/sata3/Xenium_Data_Storage_2/20250109__205605__perturb4_mc38baseline/output-XETG00341__0032973__mc38baseline_t1_AG001__20250109__205626'
dapi_path = 'morphology.ome.tif'
cellpose_training_folder = '/mnt/sata4/cellpose_training'
total_path = os.path.join(cellpose_training_folder, os.path.basename(experiment_path))

In [3]:
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 6407680645

In [ ]:

img_fpath = pathlib.Path(os.path.join(experiment_path, dapi_path))

reader = OMETIFFReader(fpath=img_fpath)

dapi, metadata, xml_metadata = reader.read()

In [5]:
dapi = np.max(dapi, axis = 0)


In [6]:
chunk_size = (1024, 1024)

In [7]:
#maxed_xenium = read_dapi_image(os.path.join(experiment_path, dapi_path), downscale_factor=1)

data_zarr = numpy_array_to_zarr(os.path.join(total_path, 'DAPI.zarr'), dapi, chunks=chunk_size)


In [8]:
data_zarr = zarr.open(os.path.join(total_path, 'DAPI.zarr'), mode='r')

## Run cellpose

Here, we use the pretrained model to perform a nuclear segmentation with cellpose.

In [9]:

mp = r'/home/amonell/piloting/2025_Segmentation_Tutorial/models/MC38_nucleus'

In [10]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
from cellpose_2d_distributed_segmentation import distributed_eval_2d

model = models.CellposeModel(pretrained_model= mp)

model_kwargs = {'gpu':True, 'pretrained_model': mp}
eval_kwargs = {'channels':[0,0],
                'diameter':int(model.diam_labels),
                'flow_threshold':0.4, 
                'cellprob_threshold':0,
                'do_3D':False
}

# define myLocalCluster parameters
cluster_kwargs = {
    'n_workers':1,    # we only have 1 gpu, so no need for more workers
    'ncpus':64,
    'memory_limit':'380GB',
    'threads_per_worker':1,
}

# run segmentation
# segments: zarr array containing labels
# boxes: list of bounding boxes around all labels
segments, boxes = distributed_eval_2d(
    input_zarr=data_zarr,
    blocksize=chunk_size,
    write_path=os.path.join(total_path, 'DAPI_segmented.zarr'),
    model_kwargs=model_kwargs,
    eval_kwargs=eval_kwargs,
    cluster_kwargs=cluster_kwargs,
)

In [13]:
segments_array = np.array(segments)

Loading in detected transcripts


In [14]:
transcripts_file = os.path.join(experiment_path, 'transcripts.parquet')
detected_tanscripts = pd.read_parquet(os.path.join(experiment_path, transcripts_file))

In [15]:
pixel_size_file = os.path.join(experiment_path, 'experiment.xenium')

In [16]:
import json
with open(pixel_size_file, 'r') as f:
    pixel_size = json.load(f)


In [17]:
global_x_pixels = detected_tanscripts['x_location']/pixel_size['pixel_size']
global_y_pixels = detected_tanscripts['y_location']/pixel_size['pixel_size']
global_z_pixels = detected_tanscripts['z_location']/pixel_size['pixel_size']


In [18]:
detected_tanscripts['global_x_pixels'] = global_x_pixels
detected_tanscripts['global_y_pixels'] = global_y_pixels
detected_tanscripts['global_z_pixels'] = global_z_pixels

In [19]:
cell_assigments = segments_array[detected_tanscripts.global_y_pixels.values.astype(int), detected_tanscripts.global_x_pixels.values.astype(int)]

In [20]:
detected_tanscripts['nucleus_id'] = cell_assigments

In [21]:
cellxgene = pd.crosstab(index=detected_tanscripts.nucleus_id, columns=detected_tanscripts.feature_name)

In [22]:
detected_tanscripts['overlaps_nucleus'] = (cell_assigments > 0).astype(int)

In [23]:
metadata = detected_tanscripts[['global_x_pixels', 'global_y_pixels', 'nucleus_id']].groupby('nucleus_id').mean()[['global_x_pixels', 'global_y_pixels']]

In [ ]:

ad = sc.AnnData(X=cellxgene.values, var=pd.DataFrame(index=cellxgene.columns), obs=pd.DataFrame(index=cellxgene.index))


In [26]:
metadata.index = metadata.index.astype(str)

In [27]:
ad.obs = ad.obs.merge(metadata, left_index=True, right_index=True, how='left')

In [28]:
ad.obsm['X_spatial'] = ad.obs[['global_x_pixels', 'global_y_pixels']].values

In [ ]:
sc.set_figure_params(dpi=400)
sc.pl.embedding(ad, basis='spatial', color='Cd8a', vmax = 5, cmap='Blues', vmin = -0.8)

In [ ]:
window_min = 10000
window_max = 11000

sub_detected_tanscripts = detected_tanscripts[(detected_tanscripts.global_y_pixels > window_min) & (detected_tanscripts.global_y_pixels < window_max)]
sub_detected_tanscripts = sub_detected_tanscripts[(sub_detected_tanscripts.global_x_pixels > window_min) & (sub_detected_tanscripts.global_x_pixels < window_max)]
# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
# Create categorical color mapping for nucleus IDs
unique_nuclei = sub_detected_tanscripts.nucleus_id.unique()
nucleus_colors = {nuc_id: f'C{i%10}' for i, nuc_id in enumerate(unique_nuclei)}
# First subplot - just the image
ax1.imshow(data_zarr[window_min:window_max, window_min:window_max], cmap='viridis')
for nuc_id in unique_nuclei:
    mask = (sub_detected_tanscripts.nucleus_id == nuc_id) & (sub_detected_tanscripts.feature_name == 'Cd8a')
    ax1.scatter(sub_detected_tanscripts[mask].global_x_pixels - window_min, 
               sub_detected_tanscripts[mask].global_y_pixels - window_min,
               c='red',
               label=f'Nucleus {nuc_id}',
               s=5, alpha=1)
ax1.set_title('DAPI + CD8A')

# Second subplot - image with scattered points
ax2.imshow(data_zarr[window_min:window_max, window_min:window_max], cmap='Reds')



# Plot each nucleus ID with its categorical color
for nuc_id in unique_nuclei:
    mask = sub_detected_tanscripts.nucleus_id == nuc_id
    ax2.scatter(sub_detected_tanscripts[mask].global_x_pixels - window_min, 
               sub_detected_tanscripts[mask].global_y_pixels - window_min,
               c=nucleus_colors[nuc_id],
               label=f'Nucleus {nuc_id}',
               s=0.5, alpha=1, linewidths=0)

ax2.set_title('Image with Nuclear Assignments')

plt.tight_layout()
plt.show()

In [31]:
ad.write(os.path.join(experiment_path, 'nucleus_segmentation_adata.h5ad'))

In [34]:
detected_tanscripts.drop(columns=['cell_id', 'fov_name', 'codeword_index', 'codeword_category', 'is_gene'], inplace=True)

In [35]:
detected_tanscripts.to_csv(os.path.join(experiment_path, "transcripts_cellpose.csv"))